In [1]:
import os
import copy
import math
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 12})
# PyTorch
import torch
torch.manual_seed(42)
import torchvision
import torchmetrics
# Laplace
from laplace import Laplace
from laplace.curvature import AsdlEF

/cluster/tufts/hugheslabkp/eharve06/miniconda3/envs/l3d_2024f_cuda12_1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import sys
sys.path.append('../src/')

%load_ext autoreload
%autoreload 2
# Importing our custom module(s)
import losses
import utils

In [3]:
def format_time(seconds):
    hours = seconds // 3600
    minutes = (seconds % 3600) // 60
    seconds = seconds % 60
    return f"{hours} hrs. {minutes} mins. {seconds} secs."

In [4]:
criterion = 'l2-zero'
dataset_directory = '/cluster/tufts/hugheslab/eharve06/CIFAR-10'
experiments_directory = '/cluster/tufts/hugheslab/eharve06/data-emphasized-ELBo/experiments/retrained_CIFAR-10_diagEF'
lr_0s = [0.1, 0.01, 0.001, 0.0001]
ns = [100, 1000, 10000, 50000]
random_states = [1001, 2001, 3001]

columns = ['criterion', 'model_name', 'n', 'random_state', 'runtime', 'train_log_marglik', 'test_acc', 'test_nll']
retrained_df = pd.DataFrame(columns=columns)

for lr_0, n, random_state in itertools.product(lr_0s, ns, random_states):
    model_name = f'{criterion}_lr_0={lr_0}_n={n}_random_state={random_state}'
    if os.path.exists(f'{experiments_directory}/{model_name}.csv'):
        temp_df = pd.read_csv(f'{experiments_directory}/{model_name}.csv')
        row = [criterion, model_name, n, random_state, temp_df['train_sec/epoch'].sum(), temp_df.train_log_marglik.values[-1], temp_df.val_or_test_acc.values[-1], temp_df.val_or_test_nll.values[-1]]
        retrained_df.loc[len(retrained_df)] = row

In [5]:
min_indices = retrained_df.groupby(['criterion', 'n', 'random_state'])['train_log_marglik'].idxmax()
retrained_df = retrained_df.loc[min_indices]

retrained_df

,criterion,model_name,n,random_state,runtime,train_log_marglik,test_acc,test_nll
12,l2-zero,l2-zero_lr_0=0.01_n=100_random_state=1001,100,1001,7008.146201,-263.259644,0.6745,1.057087
13,l2-zero,l2-zero_lr_0=0.01_n=100_random_state=2001,100,2001,7000.924933,-242.931412,0.6659,1.055820
14,l2-zero,l2-zero_lr_0=0.01_n=100_random_state=3001,100,3001,6952.630825,-223.579773,0.6846,1.024537
15,l2-zero,l2-zero_lr_0=0.01_n=1000_random_state=1001,1000,1001,4952.520816,-404.404633,0.8877,0.442697
16,l2-zero,l2-zero_lr_0=0.01_n=1000_random_state=2001,1000,2001,4963.107343,-424.987091,0.8809,0.466042
17,l2-zero,l2-zero_lr_0=0.01_n=1000_random_state=3001,1000,3001,5867.403218,-388.481537,0.8779,0.480824
18,l2-zero,l2-zero_lr_0=0.01_n=10000_random_state=1001,10000,1001,4168.477561,-1875.979858,0.9509,0.203911
19,l2-zero,l2-zero_lr_0=0.01_n=10000_random_state=2001,10000,2001,4130.892845,-2439.328857,0.9506,0.211288
20,l2-zero,l2-zero_lr_0=0.01_n=10000_random_state=3001,10000,3001,4162.006493,-2173.709717,0.9517,0.191490
21,l2-zero,l2-zero_lr_0=0.01_n=50000_random_state=1001,50000,1001,4176.966407,-146591.953125,0.9696,0.116327


In [6]:
grouped_df = retrained_df.groupby(['criterion', 'n']).agg(lambda x: tuple(x))
columns = ['test_acc', 'test_nll']
for column in columns:
    grouped_df[f'{column}_mean'] = grouped_df[column].apply(lambda item: np.mean(item))
    grouped_df[f'{column}_std'] = grouped_df[column].apply(lambda item: np.std(item))
    grouped_df[f'{column}_min'] = grouped_df[column].apply(lambda item: np.min(item))
    grouped_df[f'{column}_max'] = grouped_df[column].apply(lambda item: np.max(item))
grouped_df = grouped_df.reset_index()
grouped_df

,criterion,n,model_name,random_state,runtime,train_log_marglik,test_acc,test_nll,test_acc_mean,test_acc_std,test_acc_min,test_acc_max,test_nll_mean,test_nll_std,test_nll_min,test_nll_max
0,l2-zero,100,"(l2-zero_lr_0=0.01_n=100_random_state=1001, l2...","(1001, 2001, 3001)","(7008.146201372147, 7000.924933433533, 6952.63...","(-263.2596435546875, -242.9314117431641, -223....","(0.6744999885559082, 0.6658999919891357, 0.684...","(1.0570869313240052, 1.0558203800201416, 1.024...",0.675000,0.007642,0.6659,0.6846,1.045815,0.015054,1.024537,1.057087
1,l2-zero,1000,"(l2-zero_lr_0=0.01_n=1000_random_state=1001, l...","(1001, 2001, 3001)","(4952.520815849304, 4963.107343196869, 5867.40...","(-404.4046325683594, -424.9870910644531, -388....","(0.887700080871582, 0.8809000849723816, 0.8779...","(0.4426965748786928, 0.466041524219513, 0.4808...",0.882167,0.004100,0.8779,0.8877,0.463187,0.015696,0.442697,0.480824
2,l2-zero,10000,"(l2-zero_lr_0=0.01_n=10000_random_state=1001, ...","(1001, 2001, 3001)","(4168.477560520172, 4130.892844676971, 4162.00...","(-1875.9798583984373, -2439.328857421875, -217...","(0.9509000778198242, 0.95059996843338, 0.95170...","(0.2039114191532135, 0.21128786611557, 0.19148...",0.951067,0.000464,0.9506,0.9517,0.202230,0.008170,0.191490,0.211288
3,l2-zero,50000,"(l2-zero_lr_0=0.01_n=50000_random_state=1001, ...","(1001, 2001, 3001)","(4176.966406583786, 4114.107445716858, 4109.98...","(-146591.953125, -170956.5, -149952.875)","(0.9696000218391418, 0.9695000648498536, 0.969...","(0.1163268289327621, 0.1145831343889236, 0.115...",0.969400,0.000216,0.9691,0.9696,0.115340,0.000730,0.114583,0.116327


In [7]:
grouped_df[['criterion', 'n', 'test_acc_mean', 'test_acc_min', 'test_acc_max']]

,criterion,n,test_acc_mean,test_acc_min,test_acc_max
0,l2-zero,100,0.675000,0.6659,0.6846
1,l2-zero,1000,0.882167,0.8779,0.8877
2,l2-zero,10000,0.951067,0.9506,0.9517
3,l2-zero,50000,0.969400,0.9691,0.9696


In [8]:
grouped_df[['criterion', 'n', 'test_nll_mean', 'test_nll_min', 'test_nll_max']]

,criterion,n,test_nll_mean,test_nll_min,test_nll_max
0,l2-zero,100,1.045815,1.024537,1.057087
1,l2-zero,1000,0.463187,0.442697,0.480824
2,l2-zero,10000,0.202230,0.191490,0.211288
3,l2-zero,50000,0.115340,0.114583,0.116327


In [9]:
criterion = 'l2-sp'
dataset_directory = '/cluster/tufts/hugheslab/eharve06/CIFAR-10'
experiments_directory = '/cluster/tufts/hugheslab/eharve06/data-emphasized-ELBo/experiments/retrained_CIFAR-10_diagEF'
lr_0s = [0.1, 0.01, 0.001, 0.0001]
ns = [100, 1000, 10000, 50000]
random_states = [1001, 2001, 3001]

columns = ['criterion', 'model_name', 'n', 'random_state', 'runtime', 'train_log_marglik', 'test_acc', 'test_nll']
retrained_df = pd.DataFrame(columns=columns)

for lr_0, n, random_state in itertools.product(lr_0s, ns, random_states):
    model_name = f'{criterion}_lr_0={lr_0}_n={n}_random_state={random_state}'
    if os.path.exists(f'{experiments_directory}/{model_name}.csv'):
        temp_df = pd.read_csv(f'{experiments_directory}/{model_name}.csv')
        row = [criterion, model_name, n, random_state, temp_df['train_sec/epoch'].sum(), temp_df.train_log_marglik.values[-1], temp_df.val_or_test_acc.values[-1], temp_df.val_or_test_nll.values[-1]]
        retrained_df.loc[len(retrained_df)] = row

In [10]:
condition = (retrained_df.n==50000)&(retrained_df.criterion=='l2-sp')&(retrained_df.random_state==1001)
temp_df = retrained_df.loc[condition]
total_time = temp_df.runtime.sum()
average_time = total_time/temp_df.shape[0]
print(f'Average runtime (sec.): {format_time(average_time)}')
print(f'Total time (sec.): {format_time(total_time)}')

Average runtime (sec.): 1.0 hrs. 9.0 mins. 54.06496077775955 secs.
Total time (sec.): 4.0 hrs. 39.0 mins. 36.25984311103821 secs.


In [11]:
min_indices = retrained_df.groupby(['criterion', 'n', 'random_state'])['train_log_marglik'].idxmax()
retrained_df = retrained_df.loc[min_indices]

retrained_df

,criterion,model_name,n,random_state,runtime,train_log_marglik,test_acc,test_nll
12,l2-sp,l2-sp_lr_0=0.01_n=100_random_state=1001,100,1001,7771.870146,-230.257370,0.3405,2.302569
1,l2-sp,l2-sp_lr_0=0.1_n=100_random_state=2001,100,2001,7320.807987,-230.257629,0.3673,2.302573
14,l2-sp,l2-sp_lr_0=0.01_n=100_random_state=3001,100,3001,7323.553846,-230.256729,0.3423,2.302560
3,l2-sp,l2-sp_lr_0=0.1_n=1000_random_state=1001,1000,1001,5009.549274,-1781.315430,0.7220,1.193988
16,l2-sp,l2-sp_lr_0=0.01_n=1000_random_state=2001,1000,2001,4991.712644,-1749.486816,0.7241,1.171732
17,l2-sp,l2-sp_lr_0=0.01_n=1000_random_state=3001,1000,3001,5644.311024,-1749.390137,0.7134,1.172170
18,l2-sp,l2-sp_lr_0=0.01_n=10000_random_state=1001,10000,1001,4249.529740,-11346.691406,0.7628,0.777652
19,l2-sp,l2-sp_lr_0=0.01_n=10000_random_state=2001,10000,2001,4185.410539,-11566.898438,0.7650,0.781921
20,l2-sp,l2-sp_lr_0=0.01_n=10000_random_state=3001,10000,3001,4257.878783,-11260.216797,0.7645,0.784060
9,l2-sp,l2-sp_lr_0=0.1_n=50000_random_state=1001,50000,1001,4476.865883,-31109.111328,0.9372,0.186945


In [12]:
grouped_df = retrained_df.groupby(['criterion', 'n']).agg(lambda x: tuple(x))
columns = ['test_acc', 'test_nll']
for column in columns:
    grouped_df[f'{column}_mean'] = grouped_df[column].apply(lambda item: np.mean(item))
    grouped_df[f'{column}_std'] = grouped_df[column].apply(lambda item: np.std(item))
    grouped_df[f'{column}_min'] = grouped_df[column].apply(lambda item: np.min(item))
    grouped_df[f'{column}_max'] = grouped_df[column].apply(lambda item: np.max(item))
grouped_df = grouped_df.reset_index()
grouped_df

,criterion,n,model_name,random_state,runtime,train_log_marglik,test_acc,test_nll,test_acc_mean,test_acc_std,test_acc_min,test_acc_max,test_nll_mean,test_nll_std,test_nll_min,test_nll_max
0,l2-sp,100,"(l2-sp_lr_0=0.01_n=100_random_state=1001, l2-s...","(1001, 2001, 3001)","(7771.870146036148, 7320.807987213135, 7323.55...","(-230.2573699951172, -230.25762939453125, -230...","(0.3404999673366546, 0.3673000037670135, 0.342...","(2.302569054031372, 2.3025725662231444, 2.3025...",0.350033,0.012231,0.3405,0.3673,2.302567,0.000005,2.302560,2.302573
1,l2-sp,1000,"(l2-sp_lr_0=0.1_n=1000_random_state=1001, l2-s...","(1001, 2001, 3001)","(5009.5492742061615, 4991.712643861771, 5644.3...","(-1781.3154296875, -1749.48681640625, -1749.39...","(0.722000002861023, 0.7240999937057495, 0.7134...","(1.1939880359649662, 1.1717322761535645, 1.172...",0.719833,0.004629,0.7134,0.7241,1.179297,0.010390,1.171732,1.193988
2,l2-sp,10000,"(l2-sp_lr_0=0.01_n=10000_random_state=1001, l2...","(1001, 2001, 3001)","(4249.529740095139, 4185.4105389118195, 4257.8...","(-11346.69140625, -11566.8984375, -11260.21679...","(0.7628000378608704, 0.7649999856948853, 0.764...","(0.7776520441055297, 0.7819208739280702, 0.784...",0.764100,0.000942,0.7628,0.7650,0.781211,0.002664,0.777652,0.784060
3,l2-sp,50000,"(l2-sp_lr_0=0.1_n=50000_random_state=1001, l2-...","(1001, 2001, 3001)","(4476.865883111954, 4307.140010595322, 4162.76...","(-31109.111328125, -29757.19921875, -30299.488...","(0.9372000694274902, 0.9431999921798706, 0.943...","(0.1869448745250701, 0.1738006152153015, 0.174...",0.941400,0.002980,0.9372,0.9438,0.178508,0.005979,0.173801,0.186945


In [13]:
grouped_df[['criterion', 'n', 'test_acc_mean', 'test_acc_min', 'test_acc_max']]

,criterion,n,test_acc_mean,test_acc_min,test_acc_max
0,l2-sp,100,0.350033,0.3405,0.3673
1,l2-sp,1000,0.719833,0.7134,0.7241
2,l2-sp,10000,0.764100,0.7628,0.7650
3,l2-sp,50000,0.941400,0.9372,0.9438


In [14]:
grouped_df[['criterion', 'n', 'test_nll_mean', 'test_nll_min', 'test_nll_max']]

,criterion,n,test_nll_mean,test_nll_min,test_nll_max
0,l2-sp,100,2.302567,2.302560,2.302573
1,l2-sp,1000,1.179297,1.171732,1.193988
2,l2-sp,10000,0.781211,0.777652,0.784060
3,l2-sp,50000,0.178508,0.173801,0.186945


In [15]:
criterion = 'l2-zero'
dataset_directory = '/cluster/tufts/hugheslab/eharve06/Oxford-IIIT_Pet'
experiments_directory = '/cluster/tufts/hugheslab/eharve06/data-emphasized-ELBo/experiments/retrained_Oxford-IIIT_Pet_diagEF'
lr_0s = [0.1, 0.01, 0.001, 0.0001]
ns = [370, 3441]
random_states = [1001, 2001, 3001]

columns = ['criterion', 'model_name', 'n', 'random_state', 'train_log_marglik', 'test_acc', 'test_nll']
retrained_df = pd.DataFrame(columns=columns)

for lr_0, n, random_state in itertools.product(lr_0s, ns, random_states):
    model_name = f'{criterion}_lr_0={lr_0}_n={n}_random_state={random_state}'
    if os.path.exists(f'{experiments_directory}/{model_name}.csv'):
        temp_df = pd.read_csv(f'{experiments_directory}/{model_name}.csv')
        row = [criterion, model_name, n, random_state, temp_df.train_log_marglik.values[-1], temp_df.val_or_test_acc.values[-1], temp_df.val_or_test_nll.values[-1]]
        retrained_df.loc[len(retrained_df)] = row
    
filtered_df = retrained_df[retrained_df['train_log_marglik'].notna()]
min_indices = filtered_df.groupby(['criterion', 'n', 'random_state'])['train_log_marglik'].idxmax()
retrained_df = filtered_df.loc[min_indices]

retrained_df

,criterion,model_name,n,random_state,train_log_marglik,test_acc,test_nll
6,l2-zero,l2-zero_lr_0=0.01_n=370_random_state=1001,370,1001,-459.612122,0.884692,0.409536
7,l2-zero,l2-zero_lr_0=0.01_n=370_random_state=2001,370,2001,-460.313660,0.878255,0.420677
8,l2-zero,l2-zero_lr_0=0.01_n=370_random_state=3001,370,3001,-475.961823,0.881983,0.369925
9,l2-zero,l2-zero_lr_0=0.01_n=3441_random_state=1001,3441,1001,-1272.866211,0.932886,0.282283
10,l2-zero,l2-zero_lr_0=0.01_n=3441_random_state=2001,3441,2001,-1324.654175,0.931520,0.291258
11,l2-zero,l2-zero_lr_0=0.01_n=3441_random_state=3001,3441,3001,-1320.142212,0.930102,0.285095


In [16]:
grouped_df = retrained_df.groupby(['criterion', 'n']).agg(lambda x: tuple(x))
columns = ['test_acc', 'test_nll']
for column in columns:
    grouped_df[f'{column}_mean'] = grouped_df[column].apply(lambda item: np.mean(item))
    grouped_df[f'{column}_std'] = grouped_df[column].apply(lambda item: np.std(item))
    grouped_df[f'{column}_min'] = grouped_df[column].apply(lambda item: np.min(item))
    grouped_df[f'{column}_max'] = grouped_df[column].apply(lambda item: np.max(item))
grouped_df = grouped_df.reset_index()
grouped_df

,criterion,n,model_name,random_state,train_log_marglik,test_acc,test_nll,test_acc_mean,test_acc_std,test_acc_min,test_acc_max,test_nll_mean,test_nll_std,test_nll_min,test_nll_max
0,l2-zero,370,"(l2-zero_lr_0=0.01_n=370_random_state=1001, l2...","(1001, 2001, 3001)","(-459.6121215820313, -460.3136596679688, -475....","(0.8846922516822815, 0.8782547116279602, 0.881...","(0.4095357748509493, 0.420676869790799, 0.3699...",0.881643,0.002639,0.878255,0.884692,0.400046,0.021779,0.369925,0.420677
1,l2-zero,3441,"(l2-zero_lr_0=0.01_n=3441_random_state=1001, l...","(1001, 2001, 3001)","(-1272.8662109375, -1324.6541748046875, -1320....","(0.9328855276107788, 0.9315203428268432, 0.930...","(0.282282579809013, 0.291258049339389, 0.28509...",0.931503,0.001136,0.930102,0.932886,0.286212,0.003748,0.282283,0.291258


In [17]:
grouped_df[['criterion', 'n', 'test_acc_mean', 'test_acc_min', 'test_acc_max']]

,criterion,n,test_acc_mean,test_acc_min,test_acc_max
0,l2-zero,370,0.881643,0.878255,0.884692
1,l2-zero,3441,0.931503,0.930102,0.932886


In [18]:
grouped_df[['criterion', 'n', 'test_nll_mean', 'test_nll_min', 'test_nll_max']]

,criterion,n,test_nll_mean,test_nll_min,test_nll_max
0,l2-zero,370,0.400046,0.369925,0.420677
1,l2-zero,3441,0.286212,0.282283,0.291258


In [19]:
criterion = 'l2-sp'
dataset_directory = '/cluster/tufts/hugheslab/eharve06/Oxford-IIIT_Pet'
experiments_directory = '/cluster/tufts/hugheslab/eharve06/data-emphasized-ELBo/experiments/retrained_Oxford-IIIT_Pet_diagEF'
lr_0s = [0.1, 0.01, 0.001, 0.0001]
ns = [370, 3441]
random_states = [1001, 2001, 3001]

columns = ['criterion', 'model_name', 'n', 'random_state', 'train_log_marglik', 'test_acc', 'test_nll']
retrained_df = pd.DataFrame(columns=columns)

for lr_0, n, random_state in itertools.product(lr_0s, ns, random_states):
    model_name = f'{criterion}_lr_0={lr_0}_n={n}_random_state={random_state}'
    if os.path.exists(f'{experiments_directory}/{model_name}.csv'):
        temp_df = pd.read_csv(f'{experiments_directory}/{model_name}.csv')
        row = [criterion, model_name, n, random_state, temp_df.train_log_marglik.values[-1], temp_df.val_or_test_acc.values[-1], temp_df.val_or_test_nll.values[-1]]
        retrained_df.loc[len(retrained_df)] = row
    
filtered_df = retrained_df[retrained_df['train_log_marglik'].notna()]
min_indices = filtered_df.groupby(['criterion', 'n', 'random_state'])['train_log_marglik'].idxmax()
retrained_df = filtered_df.loc[min_indices]

retrained_df

,criterion,model_name,n,random_state,train_log_marglik,test_acc,test_nll
6,l2-sp,l2-sp_lr_0=0.01_n=370_random_state=1001,370,1001,-929.215149,0.864195,1.353605
1,l2-sp,l2-sp_lr_0=0.1_n=370_random_state=2001,370,2001,-915.322754,0.863962,1.284406
2,l2-sp,l2-sp_lr_0=0.1_n=370_random_state=3001,370,3001,-917.240967,0.872751,1.369776
3,l2-sp,l2-sp_lr_0=0.1_n=3441_random_state=1001,3441,1001,-2850.339844,0.906682,0.411867
4,l2-sp,l2-sp_lr_0=0.1_n=3441_random_state=2001,3441,2001,-2810.989258,0.906163,0.411842
5,l2-sp,l2-sp_lr_0=0.1_n=3441_random_state=3001,3441,3001,-2839.485107,0.905492,0.411417


In [20]:
grouped_df = retrained_df.groupby(['criterion', 'n']).agg(lambda x: tuple(x))
columns = ['test_acc', 'test_nll']
for column in columns:
    grouped_df[f'{column}_mean'] = grouped_df[column].apply(lambda item: np.mean(item))
    grouped_df[f'{column}_std'] = grouped_df[column].apply(lambda item: np.std(item))
    grouped_df[f'{column}_min'] = grouped_df[column].apply(lambda item: np.min(item))
    grouped_df[f'{column}_max'] = grouped_df[column].apply(lambda item: np.max(item))
grouped_df = grouped_df.reset_index()
grouped_df

,criterion,n,model_name,random_state,train_log_marglik,test_acc,test_nll,test_acc_mean,test_acc_std,test_acc_min,test_acc_max,test_nll_mean,test_nll_std,test_nll_min,test_nll_max
0,l2-sp,370,"(l2-sp_lr_0=0.01_n=370_random_state=1001, l2-s...","(1001, 2001, 3001)","(-929.2151489257812, -915.32275390625, -917.24...","(0.8641949892044067, 0.8639618158340454, 0.872...","(1.35360466836096, 1.2844058398688385, 1.36977...",0.866969,0.004089,0.863962,0.872751,1.335929,0.037026,1.284406,1.369776
1,l2-sp,3441,"(l2-sp_lr_0=0.1_n=3441_random_state=1001, l2-s...","(1001, 2001, 3001)","(-2850.33984375, -2810.9892578125, -2839.48510...","(0.9066816568374634, 0.906163454055786, 0.9054...","(0.4118665885733595, 0.4118419586150059, 0.411...",0.906112,0.000487,0.905492,0.906682,0.411708,0.000206,0.411417,0.411867


In [21]:
grouped_df[['criterion', 'n', 'test_acc_mean', 'test_acc_min', 'test_acc_max']]

,criterion,n,test_acc_mean,test_acc_min,test_acc_max
0,l2-sp,370,0.866969,0.863962,0.872751
1,l2-sp,3441,0.906112,0.905492,0.906682


In [22]:
grouped_df[['criterion', 'n', 'test_nll_mean', 'test_nll_min', 'test_nll_max']]

,criterion,n,test_nll_mean,test_nll_min,test_nll_max
0,l2-sp,370,1.335929,1.284406,1.369776
1,l2-sp,3441,0.411708,0.411417,0.411867


In [23]:
criterion = 'l2-zero'
dataset_directory = '/cluster/tufts/hugheslab/eharve06/Flowers_102'
experiments_directory = '/cluster/tufts/hugheslab/eharve06/data-emphasized-ELBo/experiments/retrained_Flowers_102_diagEF'
lr_0s = [0.1, 0.01, 0.001, 0.0001]
ns = [510, 1020]
random_states = [1001, 2001, 3001]

columns = ['criterion', 'model_name', 'n', 'random_state', 'train_log_marglik', 'test_acc', 'test_nll']
retrained_df = pd.DataFrame(columns=columns)

for lr_0, n, random_state in itertools.product(lr_0s, ns, random_states):
    model_name = f'{criterion}_lr_0={lr_0}_n={n}_random_state={random_state}'
    if os.path.exists(f'{experiments_directory}/{model_name}.csv'):
        temp_df = pd.read_csv(f'{experiments_directory}/{model_name}.csv')
        row = [criterion, model_name, n, random_state, temp_df.train_log_marglik.values[-1], temp_df.val_or_test_acc.values[-1], temp_df.val_or_test_nll.values[-1]]
        retrained_df.loc[len(retrained_df)] = row
        
filtered_df = retrained_df[retrained_df['train_log_marglik'].notna()]
min_indices = filtered_df.groupby(['criterion', 'n', 'random_state'])['train_log_marglik'].idxmax()
retrained_df = filtered_df.loc[min_indices]

retrained_df

,criterion,model_name,n,random_state,train_log_marglik,test_acc,test_nll
6,l2-zero,l2-zero_lr_0=0.01_n=510_random_state=1001,510,1001,-784.824951,0.846590,0.730909
7,l2-zero,l2-zero_lr_0=0.01_n=510_random_state=2001,510,2001,-779.792603,0.853450,0.686262
8,l2-zero,l2-zero_lr_0=0.01_n=510_random_state=3001,510,3001,-796.898254,0.858159,0.700889
9,l2-zero,l2-zero_lr_0=0.01_n=1020_random_state=1001,1020,1001,-957.297668,0.926593,0.367216
10,l2-zero,l2-zero_lr_0=0.01_n=1020_random_state=2001,1020,2001,-970.836426,0.923874,0.354911
11,l2-zero,l2-zero_lr_0=0.01_n=1020_random_state=3001,1020,3001,-960.152344,0.924812,0.363436


In [24]:
grouped_df = retrained_df.groupby(['criterion', 'n']).agg(lambda x: tuple(x))
columns = ['test_acc', 'test_nll']
for column in columns:
    grouped_df[f'{column}_mean'] = grouped_df[column].apply(lambda item: np.mean(item))
    grouped_df[f'{column}_std'] = grouped_df[column].apply(lambda item: np.std(item))
    grouped_df[f'{column}_min'] = grouped_df[column].apply(lambda item: np.min(item))
    grouped_df[f'{column}_max'] = grouped_df[column].apply(lambda item: np.max(item))
grouped_df = grouped_df.reset_index()
grouped_df

,criterion,n,model_name,random_state,train_log_marglik,test_acc,test_nll,test_acc_mean,test_acc_std,test_acc_min,test_acc_max,test_nll_mean,test_nll_std,test_nll_min,test_nll_max
0,l2-zero,510,"(l2-zero_lr_0=0.01_n=510_random_state=1001, l2...","(1001, 2001, 3001)","(-784.824951171875, -779.7926025390625, -796.8...","(0.8465895652770996, 0.853449821472168, 0.8581...","(0.730909150323709, 0.6862618204261965, 0.7008...",0.852733,0.004750,0.846590,0.858159,0.706020,0.018585,0.686262,0.730909
1,l2-zero,1020,"(l2-zero_lr_0=0.01_n=1020_random_state=1001, l...","(1001, 2001, 3001)","(-957.2976684570312, -970.83642578125, -960.15...","(0.9265932440757751, 0.9238739013671876, 0.924...","(0.3672155920624524, 0.3549106962886675, 0.363...",0.925093,0.001128,0.923874,0.926593,0.361854,0.005146,0.354911,0.367216


In [25]:
grouped_df[['criterion', 'n', 'test_acc_mean', 'test_acc_min', 'test_acc_max']]

,criterion,n,test_acc_mean,test_acc_min,test_acc_max
0,l2-zero,510,0.852733,0.846590,0.858159
1,l2-zero,1020,0.925093,0.923874,0.926593


In [26]:
grouped_df[['criterion', 'n', 'test_nll_mean', 'test_nll_min', 'test_nll_max']]

,criterion,n,test_nll_mean,test_nll_min,test_nll_max
0,l2-zero,510,0.706020,0.686262,0.730909
1,l2-zero,1020,0.361854,0.354911,0.367216


In [27]:
criterion = 'l2-sp'
dataset_directory = '/cluster/tufts/hugheslab/eharve06/Flowers_102'
experiments_directory = '/cluster/tufts/hugheslab/eharve06/data-emphasized-ELBo/experiments/retrained_Flowers_102_diagEF'
lr_0s = [0.1, 0.01, 0.001, 0.0001]
ns = [510, 1020]
random_states = [1001, 2001, 3001]

columns = ['criterion', 'model_name', 'n', 'random_state', 'train_log_marglik', 'test_acc', 'test_nll']
retrained_df = pd.DataFrame(columns=columns)

for lr_0, n, random_state in itertools.product(lr_0s, ns, random_states):
    model_name = f'{criterion}_lr_0={lr_0}_n={n}_random_state={random_state}'
    if os.path.exists(f'{experiments_directory}/{model_name}.csv'):
        temp_df = pd.read_csv(f'{experiments_directory}/{model_name}.csv')
        row = [criterion, model_name, n, random_state, temp_df.train_log_marglik.values[-1], temp_df.val_or_test_acc.values[-1], temp_df.val_or_test_nll.values[-1]]
        retrained_df.loc[len(retrained_df)] = row
        
filtered_df = retrained_df[retrained_df['train_log_marglik'].notna()]
min_indices = filtered_df.groupby(['criterion', 'n', 'random_state'])['train_log_marglik'].idxmax()
retrained_df = filtered_df.loc[min_indices]

retrained_df

,criterion,model_name,n,random_state,train_log_marglik,test_acc,test_nll
0,l2-sp,l2-sp_lr_0=0.1_n=510_random_state=1001,510,1001,-2358.463867,0.296046,4.624085
1,l2-sp,l2-sp_lr_0=0.1_n=510_random_state=2001,510,2001,-2358.495850,0.203985,4.624209
2,l2-sp,l2-sp_lr_0=0.1_n=510_random_state=3001,510,3001,-2358.385254,0.187693,4.623873
9,l2-sp,l2-sp_lr_0=0.01_n=1020_random_state=1001,1020,1001,-4229.246582,0.826620,2.602177
10,l2-sp,l2-sp_lr_0=0.01_n=1020_random_state=2001,1020,2001,-4264.004883,0.825626,2.599816
11,l2-sp,l2-sp_lr_0=0.01_n=1020_random_state=3001,1020,3001,-4304.702148,0.827271,2.610205


In [28]:
grouped_df = retrained_df.groupby(['criterion', 'n']).agg(lambda x: tuple(x))
columns = ['test_acc', 'test_nll']
for column in columns:
    grouped_df[f'{column}_mean'] = grouped_df[column].apply(lambda item: np.mean(item))
    grouped_df[f'{column}_std'] = grouped_df[column].apply(lambda item: np.std(item))
    grouped_df[f'{column}_min'] = grouped_df[column].apply(lambda item: np.min(item))
    grouped_df[f'{column}_max'] = grouped_df[column].apply(lambda item: np.max(item))
grouped_df = grouped_df.reset_index()
grouped_df

,criterion,n,model_name,random_state,train_log_marglik,test_acc,test_nll,test_acc_mean,test_acc_std,test_acc_min,test_acc_max,test_nll_mean,test_nll_std,test_nll_min,test_nll_max
0,l2-sp,510,"(l2-sp_lr_0=0.1_n=510_random_state=1001, l2-sp...","(1001, 2001, 3001)","(-2358.4638671875, -2358.495849609375, -2358.3...","(0.2960460782051086, 0.2039852440357208, 0.187...","(4.624085110481203, 4.624208929403407, 4.62387...",0.229241,0.047704,0.187693,0.296046,4.624056,0.000139,4.623873,4.624209
1,l2-sp,1020,"(l2-sp_lr_0=0.01_n=1020_random_state=1001, l2-...","(1001, 2001, 3001)","(-4229.24658203125, -4264.0048828125, -4304.70...","(0.8266200423240662, 0.8256259560585022, 0.827...","(2.602176932750397, 2.599815851441242, 2.61020...",0.826506,0.000677,0.825626,0.827271,2.604066,0.004447,2.599816,2.610205


In [29]:
grouped_df[['criterion', 'n', 'test_acc_mean', 'test_acc_min', 'test_acc_max']]

,criterion,n,test_acc_mean,test_acc_min,test_acc_max
0,l2-sp,510,0.229241,0.187693,0.296046
1,l2-sp,1020,0.826506,0.825626,0.827271


In [30]:
grouped_df[['criterion', 'n', 'test_nll_mean', 'test_nll_min', 'test_nll_max']]

,criterion,n,test_nll_mean,test_nll_min,test_nll_max
0,l2-sp,510,4.624056,4.623873,4.624209
1,l2-sp,1020,2.604066,2.599816,2.610205


In [6]:
dataset_directory = '/cluster/tufts/hugheslab/eharve06/CIFAR-10'
n = 1_000
tune = False
random_state = 1001
augmented_train_dataset, train_dataset, val_or_test_dataset = utils.get_cifar10_datasets(dataset_directory, n, tune, random_state)

Files already downloaded and verified
Files already downloaded and verified


In [7]:
batch_size = 128
num_workers = 0
augmented_train_loader = torch.utils.data.DataLoader(augmented_train_dataset, batch_size=min(batch_size, len(augmented_train_dataset)), shuffle=True, num_workers=num_workers, drop_last=True)
train_loader = torch.utils.data.DataLoader(augmented_train_dataset, batch_size=min(batch_size, len(augmented_train_dataset)), shuffle=True, num_workers=num_workers)
#train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=min(batch_size, len(train_dataset)), num_workers=num_workers)
val_or_test_loader = torch.utils.data.DataLoader(val_or_test_dataset, batch_size=batch_size, num_workers=num_workers)

In [8]:
steps = 6000
num_batches = len(augmented_train_loader)
epochs = int(steps/num_batches)
epochs

857

In [9]:
num_classes = 10
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

model = torchvision.models.resnet50()
model.fc = torch.nn.Linear(in_features=2048, out_features=num_classes, bias=True)
model.to(device)

bb_loc = torch.load('/cluster/tufts/hugheslab/eharve06/resnet50_torchvision/resnet50_torchvision_mean.pt', map_location=torch.device('cpu'), weights_only=False).to(device)
clf_loc = torch.ones((2048 * num_classes) + num_classes).to(device)

cuda:0


In [11]:
num_classes = 10
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

model = torchvision.models.vit_b_16()
model.heads.head = torch.nn.Linear(in_features=768, out_features=num_classes, bias=True)
model.to(device)

bb_loc = torch.load('/cluster/tufts/hugheslab/eharve06/vit_b_16_torchvision/vit_b_16_torchvision_mean.pt', map_location=torch.device('cpu'), weights_only=False).to(device )
clf_loc = torch.ones((768 * num_classes) + num_classes).to(device)

cuda:0


In [9]:
num_classes = 10
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

model = torchvision.models.convnext_tiny()
model.classifier[2] = torch.nn.Linear(in_features=768, out_features=num_classes, bias=True)
model.to(device)

bb_loc = torch.load('/cluster/tufts/hugheslab/eharve06/convnext_tiny_torchvision/convnext_tiny_torchvision_mean.pt', map_location=torch.device('cpu'), weights_only=False).to(device)
clf_loc = torch.ones((768 * num_classes) + num_classes).to(device)

cuda:0


In [3]:
from laplace.baselaplace import DiagLaplace

class L2SPLaplace(DiagLaplace):
    
    @property
    def prior_precision(self) -> torch.Tensor:
        return self._prior_precision

    @prior_precision.setter
    def prior_precision(self, prior_precision: float | torch.Tensor):
        self._posterior_scale = None
        self._prior_precision = prior_precision.to(
            device=self._device, dtype=self._dtype
        )
        
    @property
    def prior_precision_diag(self) -> torch.Tensor:
        alpha = self.prior_precision[0]
        beta = self.prior_precision[1]
        return torch.cat([alpha * torch.ones(self.D, device=self._device, dtype=self._dtype),
                          beta * torch.ones(self.n_params - self.D, device=self._device, dtype=self._dtype)])

prior_precision = torch.tensor([1.0, 1.0]).to(device)
prior_mean = torch.cat((bb_loc, clf_loc))
la = L2SPLaplace(
    model=model,
    likelihood='classification',
    prior_precision=prior_precision,
    prior_mean=prior_mean,
    backend=AsdlEF,
)
la.D = len(bb_loc)
alpha = la.prior_precision[0].item() / len(train_dataset)
beta = la.prior_precision[1].item() / len(train_dataset)

steps = 6000
num_batches = len(augmented_train_loader)
epochs = int(steps/num_batches)

epoch = 0

F, K, gamma = 10, 100, 1
if epoch % F == 0:
    
    print(la.prior_precision)
    la.fit(train_loader)
    la.optimize_prior_precision(
        pred_type='glm', 
        method='marglik', 
        n_steps=K, 
        lr=gamma, 
        init_prior_prec=la.prior_precision
    )
    alpha = la.prior_precision[0].item() / len(train_dataset)
    beta = la.prior_precision[1].item() / len(train_dataset)
    print(la.prior_precision)

NameError: name 'model' is not defined

In [10]:
from sklearn.model_selection import train_test_split
# Hugging Face
import datasets
import transformers

def get_news_ag_datasets(dataset_directory, n, tune, random_state):

    ag_news_dataset = datasets.load_dataset('ag_news', cache_dir=dataset_directory)
    full_train_dataset = pd.DataFrame(ag_news_dataset['train'])
    full_test_dataset = pd.DataFrame(ag_news_dataset['test'])

    if n == len(full_train_dataset):
        train_and_val_indices = np.arange(0, len(full_train_dataset))
    else:
        train_and_val_indices, _ = train_test_split(
            np.arange(0, len(full_train_dataset)), 
            test_size=None, 
            train_size=n, 
            random_state=random_state, 
            shuffle=True, 
            stratify=np.array(full_train_dataset.label),
        )
        
    val_size = int((1/5) * n)
    train_indices, val_indices = train_test_split(
        train_and_val_indices, 
        test_size=val_size, 
        train_size=n-val_size, 
        random_state=random_state, 
        shuffle=True, 
        stratify=np.array(full_train_dataset.label)[train_and_val_indices],
    )
    
    if tune:
        return datasets.DatasetDict({
            'train': datasets.Dataset.from_pandas(full_train_dataset.iloc[train_indices]),
            'val_or_test': datasets.Dataset.from_pandas(full_train_dataset.iloc[val_indices])
        })
    else:
        return datasets.DatasetDict({
            'train': datasets.Dataset.from_pandas(full_train_dataset.iloc[train_and_val_indices]),
            'val_or_test': datasets.Dataset.from_pandas(full_test_dataset)
        })


In [20]:
dataset_directory = '/cluster/tufts/hugheslab/eharve06/AG_News'
n = 400
tune = False
random_state = 1001
dataset = get_news_ag_datasets(dataset_directory, n, tune, random_state)

tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize_function(example):
    return tokenizer(example['text'], truncation=True, padding='max_length', max_length=512)

tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")

tokenized_datasets['train'].set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
tokenized_datasets['val_or_test'].set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

batch_size = 32
augmented_train_loader = torch.utils.data.DataLoader(tokenized_datasets['train'], batch_size=batch_size, shuffle=True)
train_loader = torch.utils.data.DataLoader(tokenized_datasets['train'], batch_size=batch_size)
val_or_test_loader = torch.utils.data.DataLoader(tokenized_datasets['val_or_test'], batch_size=batch_size)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

Map: 100%|██████████| 7600/7600 [00:04<00:00, 1692.32 examples/s]

cuda


In [30]:
import transformers
model = transformers.BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=4)
model.classifier

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Linear(in_features=768, out_features=4, bias=True)

In [21]:
import torch
import torch.nn as nn
import typing

class MyBERT(torch.nn.Module):
    def __init__(self, tokenizer: transformers.PreTrainedTokenizer, num_labels: int):
        super().__init__()
        config = transformers.BertConfig.from_pretrained("bert-base-uncased")
        config.pad_token_id = tokenizer.pad_token_id
        config.num_labels = num_labels
        self.hf_model = transformers.BertForSequenceClassification.from_pretrained(
            "bert-base-uncased", config=config
        )

    def forward(self, data: typing.MutableMapping) -> torch.Tensor:
        device = next(self.parameters()).device
        input_ids = data["input_ids"].to(device)
        attn_mask = data["attention_mask"].to(device)
        output_dict = self.hf_model(input_ids=input_ids, attention_mask=attn_mask)
        return output_dict.logits
    
num_classes = 4
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')
model = MyBERT(tokenizer, num_classes)
    
la = DiagLaplace(
    model=model,
    likelihood='classification',
    prior_precision=1.0,
    backend=AsdlEF,
)

epoch = 0

F, K, gamma = 10, 100, 1
if epoch % F == 0:
    
    print(la.prior_precision)
    la.fit(train_loader)
    la.optimize_prior_precision(
        pred_type='glm', 
        method='marglik', 
        n_steps=K, 
        lr=gamma, 
        init_prior_prec=la.prior_precision
    )
    print(la.prior_precision)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tensor([1.])


/cluster/tufts/hugheslabkp/eharve06/miniconda3/envs/l3d_2024f_cuda12_1/lib/python3.12/site-packages/torch/nn/modules/module.py:1640: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  self._maybe_warn_non_full_backward_hook(args, result, grad_fn)
/cluster/tufts/hugheslabkp/eharve06/miniconda3/envs/l3d_2024f_cuda12_1/lib/python3.12/site-packages/laplace/baselaplace.py:435: UserWarning: By default `link_approx` is `probit`. Make sure to set it equals to the way you want to call `la(test_data, pred_type=..., link_approx=...)`.
  warnings.warn(


tensor([0.3453])


In [28]:
from asdl.precondition import PreconditioningConfig, KfacGradientMaker

model = torchvision.models.resnet50()
model = torchvision.models.vit_b_16()
#model = torchvision.models.convnext_tiny()
config = PreconditioningConfig(data_size=128, damping=0.01)
gm = KfacGradientMaker(model, config, fisher_type="fisher_emp")

AssertionError: Not supported module: NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)

In [38]:
model = torchvision.models.resnet50()
#model = torchvision.models.vit_b_16()
#model = torchvision.models.convnext_tiny()

supported_layers = ['Linear', 'Conv2d', 'BatchNorm2d', 'LayerNorm', 
                    'LayerNorm2d', 'CNBlock', 'MultiheadAttention', 'Embedding']
# Note: VisionTransformer has a class_token, Encoder has a pos_embedding, and
#       NonDynamicallyQuantizableLinear is handled in MultiheadAttention
excluded_layers = ['VisionTransformer', 'Encoder', 
                   'NonDynamicallyQuantizableLinear']

for module in model.modules():
    module_requires_grad = any(p.requires_grad for p in module.parameters(recurse=False))
    module_name = str(module).partition('(')[0]
    if module_requires_grad and (module_name not in supported_layers and module_name not in excluded_layers):
        print(module_name)
        print([name for name, param in module.named_parameters(recurse=False)])